In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("data/iris.csv")

In [3]:
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
X=data.drop(['Id','Species'],axis=1)
y=data['Species']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=30, stratify=y)

In [6]:
from sklearn.svm import SVC  #SVC - Supoprt Vector Classifier
cls=SVC()

In [7]:
from sklearn.model_selection import GridSearchCV
parameters = {'C':[0.001,0.1,10], 'gamma':[0.1,0.01],'kernel':['rbf','linear']}

#C - Regularization - squared l2 error is calculated
#gamma - Fit of the model
#kernal - kernal to be used 

In [8]:
grid=GridSearchCV(cls, parameters,cv=5)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.001, 0.1, 10], 'gamma': [0.1, 0.01],
                         'kernel': ['rbf', 'linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [9]:
#to get the best parameter for the model
grid.best_params_

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

In [10]:
svclassifier=SVC(C=10,gamma=0.1,kernel='rbf',probability=True)
svclassifier.fit(X_train,y_train)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [11]:
ypred=svclassifier.predict(X_test)

In [12]:
#accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test,ypred)

1.0

In [13]:
#confusison matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,ypred)

array([[10,  0,  0],
       [ 0, 10,  0],
       [ 0,  0, 10]], dtype=int64)

In [14]:
from sklearn.metrics import roc_auc_score
rfc_probs = svclassifier.predict_proba(X_test)
rfc_probs

array([[0.01664785, 0.0253024 , 0.95804975],
       [0.00891223, 0.02092208, 0.97016568],
       [0.96483987, 0.02442208, 0.01073805],
       [0.0148677 , 0.97745987, 0.00767244],
       [0.01672457, 0.97873298, 0.00454245],
       [0.97487168, 0.0152383 , 0.00989003],
       [0.97162014, 0.01769128, 0.01068858],
       [0.96820001, 0.02122347, 0.01057652],
       [0.016428  , 0.02752043, 0.95605157],
       [0.01054423, 0.93080941, 0.05864636],
       [0.00833252, 0.98229884, 0.00936864],
       [0.01003509, 0.00947404, 0.98049088],
       [0.00799491, 0.98418695, 0.00781813],
       [0.01114102, 0.00323849, 0.98562049],
       [0.96863992, 0.02111573, 0.01024435],
       [0.96434763, 0.02402558, 0.01162679],
       [0.93075579, 0.05583702, 0.01340719],
       [0.01761087, 0.55131053, 0.43107861],
       [0.97116862, 0.01897663, 0.00985475],
       [0.01146684, 0.71839384, 0.27013931],
       [0.02269909, 0.17689084, 0.80041008],
       [0.96186372, 0.02712373, 0.01101255],
       [0.

In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_test,ypred))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      1.00      1.00        10
 Iris-virginica       1.00      1.00      1.00        10

       accuracy                           1.00        30
      macro avg       1.00      1.00      1.00        30
   weighted avg       1.00      1.00      1.00        30



In [16]:
import warnings
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
def versiontuple(version):
    return tuple(map(int, (version.split("."))))
def decision_plot(X, y, classifier, test_idx=None, resolution=0.02):
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'green', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    # plot the decision surface
    x1min, x1max = X['SepalLengthCm'].min() - 1, X['SepalLengthCm'].max() + 1
    x2min, x2max = X['SepalWidthCm'].min() - 1, X['SepalWidthCm'].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1min, x1max, resolution),
                           np.arange(x2min, x2max, resolution))
    #Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = classifier.predict(X)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=cmap(idx),
                    marker=markers[idx], label=cl)